# ======================== Library ============================

In [1]:
from __future__ import absolute_import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary

from PIL import Image
import scipy.io as sio
import numpy as np
import os

import skimage.measure
import pickle

from sklearn.metrics import precision_recall_curve,auc
import matplotlib.pyplot as plt
from matplotlib import gridspec

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

import scipy.io as sio

from pptx import Presentation
from pptx.enum.text import PP_ALIGN
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Inches, Pt,Cm

# ======================== Functions ============================

In [2]:
def read_files():
    # Upload the data 
    return image_train, image_test, label_train, label_test

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1      = nn.Conv2d(3, 32, kernel_size=3)
        self.conv1_Bn   = nn.BatchNorm2d(32)
        self.relu_1     = nn.ReLU()
        self.max_pool_1  = nn.MaxPool2d(2)
        
        self.conv2      = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_Bn   = nn.BatchNorm2d(64)
        self.relu_2     = nn.ReLU()
        
        
        self.conv3      = nn.Conv2d(64, 64, kernel_size=3)
        self.conv3_Bn   = nn.BatchNorm2d(64)
        self.relu_3     = nn.ReLU()
        self.max_pool_3  = nn.MaxPool2d(2)
        
        self.conv4      = nn.Conv2d(64, 128, kernel_size=3)
        self.conv4_Bn   = nn.BatchNorm2d(128)
        self.relu_4     = nn.ReLU()
        self.max_pool_4  = nn.MaxPool2d(2)
        
        self.conv5      = nn.Conv2d(128, 256, kernel_size=3)
        self.conv5_Bn   = nn.BatchNorm2d(256)
        self.relu_5     = nn.ReLU()
        
        self.fc1        = nn.Linear(2304,128)
        self.fc2        = nn.Linear(128,10)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu_1(x)
        x = self.conv1_Bn(x)
        x = self.max_pool_1(x)
        
        x = self.conv2(x)
        x = self.relu_2(x)
        x = self.conv2_Bn(x)
        
        x = self.conv3(x)
        x = self.relu_3(x)
        x = self.conv3_Bn(x)
        x = self.max_pool_3(x)
        
        x = self.conv4(x)
        x = self.relu_4(x)
        x = self.conv4_Bn(x)
        x = self.max_pool_4(x)
        
        x = self.conv5(x)
        x = self.relu_5(x)
        x = self.conv5_Bn(x)
        
        #print(x.shape)
        x = x.view(-1,2304)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(x,dim=1)
        
        return x

In [5]:
class GuidedBackprop():
    """
       Produces gradients generated with guided back propagation from the given image
    """
    def __init__(self, model,):
        self.model = model
        self.gradients = None
        self.forward_relu_outputs = []
        # Put model in evaluation mode
        self.model.eval()
        self.update_relus()
        self.hook_layers()

    def hook_layers(self):
        def hook_function(module, grad_in, grad_out):
            self.gradients = grad_in[0]
        # Register hook to the first layer
        first_layer = list(self.model._modules.items())[0][1]
        first_layer.register_backward_hook(hook_function)

    def update_relus(self):
        """
            Updates relu activation functions so that
                1- stores output in forward pass
                2- imputes zero for gradient values that are less than zero
        """
        def relu_backward_hook_function(module, grad_in, grad_out):
            """
            If there is a negative gradient, change it to zero
            """
            # Get last forward output
            corresponding_forward_output = self.forward_relu_outputs[-1]
            corresponding_forward_output[corresponding_forward_output > 0] = 1
            modified_grad_out = corresponding_forward_output * torch.clamp(grad_in[0], min=0.0)
            del self.forward_relu_outputs[-1]  # Remove last forward output
            return (modified_grad_out,)

        def relu_forward_hook_function(module, ten_in, ten_out):
            """
            Store results of forward pass
            """
            self.forward_relu_outputs.append(ten_out)

        # Loop through layers, hook up ReLUs
        for pos, module in model._modules.items():
            if isinstance(module, torch.nn.modules.activation.ReLU):
                print(pos)
                module.register_backward_hook(relu_backward_hook_function)
                module.register_forward_hook(relu_forward_hook_function)

    def generate_gradients(self, input_image, target_class, cnn_layer, filter_pos):
        
        # Forward pass
        x = input_image
        for index, layer in self.model._modules.items():
            x = layer(x)
            if index == cnn_layer:
                break
                
        # Backward pass
        self.model.zero_grad()
        conv_output = torch.sum(torch.abs(x[0, filter_pos]))
        conv_output.backward()
        
        #convert to numpy
        gradients   = self.gradients.cpu()
        gradients_as_arr = gradients.data.numpy()[0]
        return gradients_as_arr

In [6]:
def get_positive_negative_saliency(gradient):
    """
        Generates positive and negative saliency maps based on the gradient
    Args:
        gradient (numpy arr): Gradient of the operation to visualize
    returns:
        pos_saliency ( )
    """
    pos_saliency = (np.maximum(0, gradient) / gradient.max())
    neg_saliency = (np.maximum(0, -gradient) / -gradient.min())
    
    pos_saliency = np.moveaxis(pos_saliency,0,2)
    neg_saliency = np.moveaxis(neg_saliency,0,2)
    return pos_saliency, neg_saliency

In [7]:
def move_to_gpu(s,e,flag):
    if flag==0:
        image = image_train[s:e].to(device)
        label = label_train[s:e].to(device)
    else:
        image  = image_test[s:e].to(device)
        label  = label_test[s:e].to(device)    
    
    return image, label 

In [8]:
def forward_images(images,cnn_layer):
    batchsize  = 50
    images_out = []
    l = [(x,x+batchsize) for x in range(0,images.shape[0],batchsize)]
    for s,e in l:
        image_train1, label_train1 = move_to_gpu(s,e,0)
        for index, layer in model._modules.items():
            image_train1 = layer(image_train1)
            if index == cnn_layer:
                break
        image_train1 = image_train1.cpu().data.numpy()
        images_out.append(image_train1)
    images_out = np.concatenate(images_out,axis=0)
    images_out = np.moveaxis(images_out,1,-1)
    return images_out

In [9]:
imagesdef Node_Best_images(images_out,percentile):
    
    outputs    = images_out.reshape((images_out.shape[0],images_out.shape[1]*images_out.shape[2],images_out.shape[3]))
    mat_1      = np.zeros((outputs.shape[0],outputs.shape[2]))
    for i in range(outputs.shape[2]):
        mat        = outputs[:,:,i]
        ind        = int(round(percentile*outputs.shape[1]))
        a          = -1*np.sort(-mat,axis=1)
        b          = a[:,:ind]
        c          = np.mean(b,axis=1)
        mat_1[:,i] = c
            
    filter_images = np.argsort(-mat_1,axis=0)
    mat_2         = -1*np.sort(-mat_1,axis=0)
    mat_3         = np.mean(mat_2[:9,:],axis=0)
    mat_4         = np.argsort(-mat_3)
    return filter_images,mat_1,mat_4

In [10]:
def display(filters_list,images,images_out,ss):
    
    fig     = plt.figure(figsize=(10, 20))
    columns = 2
    rows    = len(filters_list)
    m       = 1
    for k in filters_list:
        
        inds = filter_images[:9,k]
        
        rw_size    = 64
        cl_size    = 64
        margin     = 5
        results_1  = np.zeros((3 * rw_size + 2 * margin, 3 * cl_size + 2 * margin,1))
        
        for i in range(3):
            for j in range(3):    

                #filter_img       = images_out[inds[j + (i * 3)],:,:,k]
                im_as_ten        = images[inds[j + (i * 3)],:,:,:]
                im_as_ten.unsqueeze_(0)
                prep_img         = Variable(im_as_ten, requires_grad=True).to(device)
                target_class     = label_train[j + (i * 3)]
                guided_grads     = GBP.generate_gradients(prep_img, target_class, cnn_layer, k)
                guided_grads_1   = np.moveaxis(guided_grads,0,2)
                guided_grads_1   = (guided_grads_1 -guided_grads_1.min())/(guided_grads_1.max()-guided_grads_1.min())
                guided_grads_1   = np.expand_dims(np.sum(guided_grads_1,axis=2),axis=2)
                
                horizontal_start = i * rw_size + i * margin
                horizontal_end   = horizontal_start + rw_size
                vertical_start   = j * cl_size + j * margin
                vertical_end     = vertical_start + cl_size

                results_1[horizontal_start: horizontal_end, vertical_start: vertical_end,:] = guided_grads_1
        
        fig.add_subplot(rows, columns, m)
        fig.subplots_adjust(wspace=0.3, hspace=0.3)
        plt.imshow(results_1[...,0],cmap="gray")
        s = "Node: "+str(k)+" outputs"+"\n"+np.array2string(label_train[inds].numpy(), precision=0, separator=',',suppress_small=True)+"\n"+np.array2string(response[inds,k], precision=2, separator=',',suppress_small=True)
        plt.title(s)
        m = m+1
        
        rw_size    = 64
        cl_size    = 64
        margin     = 5
        results_2  = np.zeros((3 * rw_size + 2 * margin, 3 * cl_size + 2 * margin,3))
        for i in range(3):
            for j in range(3):
                
                filter_img       = images[inds[j + (i * 3)],:,:,:].numpy()
                filter_img       = np.moveaxis(filter_img,0,-1)
                
                horizontal_start = i * rw_size + i * margin
                horizontal_end   = horizontal_start + rw_size
                vertical_start   = j * cl_size + j * margin
                vertical_end     = vertical_start + cl_size

                results_2[horizontal_start: horizontal_end, vertical_start: vertical_end,:] = filter_img
                
        fig.add_subplot(rows, columns, m)
        fig.subplots_adjust(wspace=0.3, hspace=0.3)
        plt.imshow(results_2)
        s = "Node: "+str(k)+" outputs"+"\n"+np.array2string(label_train[inds].numpy(), precision=0, separator=',',suppress_small=True)+"\n"+np.array2string(response[inds,k], precision=2, separator=',',suppress_small=True)
        plt.title(s)
        m = m+1
    plt.savefig(ss)
    plt.show()

In [11]:
def GenSlide(Slide,Title,Path_1,Path_2,Path_3):
    title_placeholder      = slide.shapes.title
    title_placeholder.text = Title
    shapes                 = slide.shapes
    
    picture_1 = shapes.add_picture(Path_1,Cm(1),Cm(0))
    
    picture_1.crop_left   = 0.035
    picture_1.crop_right  = 0.09
    picture_1.crop_top    = 0.055
    picture_1.crop_bottom = 0.01

    picture_1.height = Cm(18)
    picture_1.width  = Cm(8)
    
    picture_2= shapes.add_picture(Path_2,Cm(9),Cm(0))
    
    picture_2.crop_left   = 0.035
    picture_2.crop_right  = 0.09
    picture_2.crop_top    = 0.055
    picture_2.crop_bottom = 0.01
    
    picture_2.height = Cm(18)
    picture_2.width  = Cm(8)
    
    picture_3= shapes.add_picture(Path_3,Cm(17),Cm(0))
    
    picture_3.crop_left   = 0.035
    picture_3.crop_right  = 0.09
    picture_3.crop_top    = 0.055
    picture_3.crop_bottom = 0.01
    
    picture_3.height = Cm(18)
    picture_3.width  = Cm(8)
    
    return Slide

# ===================== upload model ==========================

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = torch.load(r"models\model.pth").to(device)
model.eval()
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 62, 62]             896
              ReLU-2           [-1, 32, 62, 62]               0
       BatchNorm2d-3           [-1, 32, 62, 62]              64
         MaxPool2d-4           [-1, 32, 31, 31]               0
            Conv2d-5           [-1, 64, 29, 29]          18,496
              ReLU-6           [-1, 64, 29, 29]               0
       BatchNorm2d-7           [-1, 64, 29, 29]             128
            Conv2d-8           [-1, 64, 27, 27]          36,928
              ReLU-9           [-1, 64, 27, 27]               0
      BatchNorm2d-10           [-1, 64, 27, 27]             128
        MaxPool2d-11           [-1, 64, 13, 13]               0
           Conv2d-12          [-1, 128, 11, 11]          73,856
             ReLU-13          [-1, 128, 11, 11]               0
      BatchNorm2d-14          [-1, 128,

# ====================== Upload data ========================

In [13]:
image_train, image_test, label_train, label_test = read_files()

In [16]:
image_train = np.array(image_train).astype('float32')
image_test  = np.array(image_test).astype('float32')

image_train = np.moveaxis(image_train,-1,1)
image_test  = np.moveaxis(image_test,-1,1)

image_train = torch.from_numpy(image_train / 255.0)
image_test  = torch.from_numpy(image_test / 255.0)
label_train = torch.from_numpy(label_train)
label_test  = torch.from_numpy(label_test)

In [17]:
cnn_layer  = "relu_4"
images_out = forward_images(image_train,cnn_layer)

In [34]:
filter_images,response,node_order = Node_Best_images(images_out,0.8)

# ================ Display in presentation =======================

In [35]:
SLD_SECTION                  = 0
SLD_LAYOUT_TITLE_AND_CONTENT = 1
prs                          = Presentation()
slide_layout                 = prs.slide_layouts[SLD_SECTION]

In [36]:
GBP = GuidedBackprop(model)

relu_1
relu_2
relu_3
relu_4
relu_5


In [ ]:
l = [x for x in range(0,filter_images.shape[1],3)]
for i, x in enumerate(l):
    s = str(i%3)+".jpg"
    
    if x == 126:
            filters_list = [node_order[126],node_order[127]]
    else:
        filters_list = [node_order[x],node_order[x+1],node_order[x+2]]
        
    display(filters_list,image_train,images_out,s)
    
    if i%3==2:
        
        slide = prs.slides.add_slide(slide_layout)
        Slide = GenSlide(slide,"Title","0.jpg","1.jpg","2.jpg")
prs.save(r'prsentation\presentation80.pptx') 